In [1]:
# %matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import requests

# Access cleaned housing data (using Power Query in Excel)

metro_hv_df = pd.read_csv("Data/Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month_modified.csv")
metro_inventory_df = pd.read_csv("Data/Metro_invt_fs_uc_sfr_sm_month_modified.csv")
zipcode_hv_df = pd.read_csv("Data/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month_modified.csv")
census_df = pd.read_csv("Data/Census/all_census_data_zipcode.csv")

# Merge house value file with census file
hv_census_df = pd.merge(zipcode_hv_df, census_df, how='outer', on='Zipcode_Year')

# Drop zipcodes that don't have exist in one of the files
hv_census_df = hv_census_df.dropna()

# Adding affordability column (how many years of household income would it take to buy a house at the average house price)
hv_census_df["Affordability"] = hv_census_df["Average House Price"] / hv_census_df["Household Income"]

hv_census_df = hv_census_df.drop(columns=['Zipcode_y', 'Year_y'])
hv_census_df = hv_census_df.rename(columns={"Zipcode_x": "Zipcode", "Year_x": "Year"})
hv_census_df["Zipcode"] = hv_census_df["Zipcode"].astype(int)
hv_census_df["Year"] = hv_census_df["Year"].astype(int)
hv_census_df["Population"] = hv_census_df["Population"].astype(int)
hv_census_df["Household Income"] = hv_census_df["Household Income"].astype(int)
hv_census_df["Per Capita Income"] = hv_census_df["Per Capita Income"].astype(int)
hv_census_df["Poverty Count"] = hv_census_df["Poverty Count"].astype(int)
hv_census_df["Average House Price"] = hv_census_df["Average House Price"].round(2)
hv_census_df["Affordability"] = hv_census_df["Affordability"].round(2)

In [2]:
metro_hv_df

,Year,Average House Price
0,2013,150641.2500
1,2014,169061.1667
2,2015,179508.3333
3,2016,191574.5000
4,2017,205885.2500
5,2018,224960.7500
6,2019,239677.0833
7,2020,254027.1667
8,2021,281973.3333


In [3]:
metro_inventory_df

,Month,Inventory Volume,Month Year,Month.1,Year,YearMonth
0,1/31/2018,27491,Jan 2018,1,2018,201801
1,2/28/2018,26389,Feb 2018,2,2018,201802
2,3/31/2018,27430,Mar 2018,3,2018,201803
3,4/30/2018,28736,Apr 2018,4,2018,201804
4,5/31/2018,30231,May 2018,5,2018,201805
5,6/30/2018,31503,Jun 2018,6,2018,201806
6,7/31/2018,32725,Jul 2018,7,2018,201807
7,8/31/2018,33643,Aug 2018,8,2018,201808
8,9/30/2018,33538,Sep 2018,9,2018,201809
9,10/31/2018,33288,Oct 2018,10,2018,201810


In [4]:
zipcode_hv_df

,Zipcode,Year,Average House Price,Zipcode_Year
0,30349,2013,80892.08333,30349_2013
1,30349,2014,98395.41667,30349_2014
2,30349,2015,106951.41670,30349_2015
3,30349,2016,116409.83330,30349_2016
4,30349,2017,129494.58330,30349_2017
...,...,...,...,...
1847,30289,2017,227717.25000,30289_2017
1848,30289,2018,248117.66670,30289_2018
1849,30289,2019,265698.83330,30289_2019
1850,30289,2020,274646.16670,30289_2020


In [5]:
census_df

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Year,Zipcode_Year
0,30002,6123,36.8,50174,35407,1401,2013,30002_2013
1,30002,5925,37.8,47297,35901,1249,2014,30002_2014
2,30002,5835,39.8,47386,38291,1134,2015,30002_2015
3,30002,5866,40.5,48403,38995,1243,2016,30002_2016
4,30002,5669,42.5,56750,43538,928,2017,30002_2017
...,...,...,...,...,...,...,...,...
1221,30680,40151,34.3,49939,20430,6790,2015,30680_2015
1222,30680,41082,34.7,49705,20949,6595,2016,30680_2016
1223,30680,42196,35.4,53263,21919,6068,2017,30680_2017
1224,30680,42975,35.1,55156,22398,5686,2018,30680_2018


In [6]:
hv_census_df

,Zipcode,Year,Average House Price,Zipcode_Year,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Affordability
0,30349,2013,80892.08,30349_2013,68941,31.8,43498,21145,12508,1.86
1,30349,2014,98395.42,30349_2014,70414,31.9,43379,21164,14694,2.27
2,30349,2015,106951.42,30349_2015,70334,32.9,44987,21254,13927,2.38
3,30349,2016,116409.83,30349_2016,69996,33.0,45573,22093,12907,2.55
4,30349,2017,129494.58,30349_2017,73310,32.6,46998,23245,12650,2.76
...,...,...,...,...,...,...,...,...,...,...
1845,30289,2015,210094.42,30289_2015,63,63.5,25625,16537,5,8.20
1846,30289,2016,212571.17,30289_2016,59,63.6,25000,16507,5,8.50
1847,30289,2017,227717.25,30289_2017,38,67.0,22250,21897,4,10.23
1848,30289,2018,248117.67,30289_2018,276,35.2,153452,36223,10,1.62
